In [ ]:
library(ggpubr)
library(ggplot2)
library(ggtext)
library(tidyr)
library(dplyr)
library(ggh4x)
library(cowplot)
library(Seurat)
library(ComplexHeatmap)

library(ggalluvial)

setwd("/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/merge/april_baff_signaling/")

In [ ]:
# load pseudobulk objects
mye = readRDS('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/merge/no_harmony/Mye/pseudobulk_mye.rds')
b = readRDS('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/merge/no_harmony/B/pseudobulk_B.rds')
pc = readRDS('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/merge/no_harmony/PC/pseudobulk_PC.rds')

In [ ]:
head(mye@meta.data)
head(b@meta.data)
head(pc@meta.data)

In [ ]:
Idents(b) <- 'subset'
b <- RenameIdents(b,
                  'Pro/PreB'='EarlyB',
                  'ImmatureB' = 'EarlyB',
                  'TransitionalB'='EarlyB',
                  'NaiveB' = 'MatureB',
                  'MemoryB' = 'MatureB')
b$stage = Idents(b)
head(b@meta.data)

In [ ]:
Idents(b) <- 'subset'
nvb <- subset(b, idents='NaiveB')
memb <- subset(b, idents='MemoryB')

In [ ]:
bmeta <- b@meta.data

paired_upns <- bmeta %>%
  distinct(UPN, Collection) %>%        # unique UPN–Collection combos
  group_by(UPN) %>%
  filter(all(c("NDMM","PT") %in% Collection)) %>%   # UPN must contain both
  pull(UPN) %>%
  unique()

length(paired_upns)
nvb_paired <- subset(nvb, subset = UPN %in% paired_upns)
memb_paired <- subset(memb, subset = UPN %in% paired_upns)
pc_paired <- subset(pc, subset = UPN %in% paired_upns)


In [ ]:
genes <- c( "TNFRSF13C", "TNFRSF13B", "TNFRSF17")

pdf("paired_violin_plots_ligand_matureB_PC.pdf", width = 3, height = 4)
for(g in genes){
    df <- FetchData(nvb_paired, vars = c(g, "Collection", "UPN"))
    colnames(df)[1] <- "expr"
    df <- df %>% filter(UPN %in% paired_upns)

    p <- ggplot(df, aes(Collection, expr, fill = Collection)) +
        geom_violin(alpha = 0.7) +
        geom_jitter(width = 0.15, size = 1, alpha = 0.6) +
        geom_line(aes(group = UPN), color = "gray40", alpha = 0.8, size = 0.25) +
        theme_bw() +
        ggtitle(paste0(g, " naive B"))

    print(p)
}

for(g in genes){
    df <- FetchData(memb_paired, vars = c(g, "Collection", "UPN"))
    colnames(df)[1] <- "expr"
    df <- df %>% filter(UPN %in% paired_upns)

    p <- ggplot(df, aes(Collection, expr, fill = Collection)) +
        geom_violin(alpha = 0.7) +
        geom_jitter(width = 0.15, size = 1, alpha = 0.6) +
        geom_line(aes(group = UPN), color = "gray40", alpha = 0.8, size = 0.25) +
        theme_bw() +
        ggtitle(paste0(g, " memory B"))

    print(p)
}
for(g in genes){
    df <- FetchData(pc_paired, vars = c(g, "Collection", "UPN"))
    colnames(df)[1] <- "expr"
    df <- df %>% filter(UPN %in% paired_upns)

    p <- ggplot(df, aes(Collection, expr, fill = Collection)) +
        geom_violin(alpha = 0.7) +
        geom_jitter(width = 0.15, size = 1, alpha = 0.6) +
        geom_line(aes(group = UPN), color = "gray40", alpha = 0.8, size = 0.25) +
        theme_bw() +
        ggtitle(paste0(g, " PC"))

    print(p)
}


dev.off()

In [ ]:
b_palette = c(
    "Pro/PreB"='#adede7',
    "ImmatureB"= '#17d4ff', 
    "TransitionalB"= '#a2c6eb',
    "NaiveB"= '#5fa5ed',
    "MemoryB"='#032563'
)

b$subset <- factor(b$subset, levels=names(b_palette))
b_exp <- AverageExpression(subset(b, subset=Collection %in% c('NBM', 'NDMM', 'PT')), features=genes, group.by=c('subset', 'Collection'))$RNA

pc_exp <- AverageExpression(subset(pc, subset=Collection %in% c('NBM', 'NDMM', 'PT')), features=genes, group.by=c('Collection'))$RNA

bpc_exp <- cbind(b_exp, pc_exp)
bpc_exp_scaled_gene <- t(scale(t(bpc_exp), center = TRUE, scale = TRUE))
bpc_exp_scaled_gene

In [ ]:
p <- Heatmap(
  as.matrix(bpc_exp_scaled_gene[c(3,2,1),]),
  name = "Z-score",
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  show_row_names = TRUE,
  show_column_names = TRUE,
  column_names_rot = 90,
  heatmap_legend_param = list(title = "Expression (z)")
)
pdf("heatmap_receptors_B_PC_scaledAvgExpr.pdf", width=6, height=4)
draw(p)
dev.off()

In [ ]:
lig <- c( "TNFSF13", "TNFSF13B")
mye_exp <- AverageExpression(subset(mye, subset=(Collection %in% c('NBM', 'NDMM', 'PT') & !subset %in% c('HSPC', 'MSC'))), features=lig, group.by=c('subset', 'Collection'))$RNA
mye_exp_scaled_gene <- t(scale(t(mye_exp), center = TRUE, scale = TRUE))
mye_exp_scaled_gene

In [ ]:
col_fun <- circlize::colorRamp2(
  c(-4, 0, 4),                    # breaks (min → mid → max)
  c("blue", "white", "red")       # colors
)

p <- Heatmap(
  as.matrix(mye_exp_scaled_gene),
  name = "Z-score",
  col=col_fun,
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  show_row_names = TRUE,
  show_column_names = TRUE,
  column_names_rot = 90,
  heatmap_legend_param = list(title = "Expression (z)")
)
    
pdf("heatmap_ligands_mye_scaledAvgExpr.pdf", width=6, height=4)
draw(p)
dev.off()

In [ ]:
# plot AUC of receptor expression
auc <- read.table('auc_normalized.tsv', sep="\t", header=T, row.names=1)
head(auc)

In [ ]:
ct_palette = c(
 'CD14 Mc'='#64d941',
 'CD16 Mc / TAM'= '#268f07',
 'Neutrophil'= '#95ad74',
 'cDC'= '#3bff8c',
 'Immature B'='#17d4ff',
 'Memory B'='#032563',
 'Naive B'='#5fa5ed',
 'Pro/Pre B'= '#adede7',
 'Transitional B'='#a2c6eb',
 'PC'='#ffbafd')

df <- auc %>% filter(gene == "TNFRSF17", Collection %in% c("NBM", "NDMM", "PT"), subset %in% c('Transitional B', 'Naive B', 'Memory B', 'PC'))

dfa <- df %>%
  group_by(subset, Collection) %>%
  summarise(AUC = mean(AUC_norm), .groups = "drop")
dfa$subset = factor(dfa$subset, levels=c('Transitional B', 'Naive B', 'Memory B', 'PC'))

p <- ggplot(dfa,
  aes(x = Collection, 
      stratum  = subset,              
      alluvium = subset,              
      y = AUC,                 
      fill  = subset,
      label = subset)) +
  geom_flow(stat = "alluvium",
            lode.guidance = "frontback",
            alpha = 0.6) +
  scale_fill_manual(values=ct_palette)+
  geom_stratum() +
  scale_x_discrete(limits = c("NBM", "NDMM", "PT")) +
  labs(
    x = "Collection",
    y = "AUC (TNFRSF17)",
    title = "TNFRSF17 AUC across Collection by subset"
  ) +
  theme_classic() +
  theme(
    legend.position = "none",
    axis.text.x = element_text(angle = 90, hjust = 1)
  )

pdf('TNFRSF17_AUC_alluvial.pdf', width=4)
print(p)
dev.off()

In [ ]:
df <- auc %>% filter(gene == "TNFRSF13B", Collection %in% c("NBM", "NDMM", "PT"), subset %in% c('Transitional B', 'Naive B', 'Memory B', 'PC'))

dfa <- df %>%
  group_by(subset, Collection) %>%
  summarise(AUC = mean(AUC_norm), .groups = "drop")
dfa$subset = factor(dfa$subset, levels=c('Transitional B', 'Naive B', 'Memory B', 'PC'))

p <- ggplot(dfa,
  aes(x = Collection, 
      stratum  = subset,              
      alluvium = subset,              
      y = AUC,                 
      fill  = subset,
      label = subset)) +
  geom_flow(stat = "alluvium",
            lode.guidance = "frontback",
            alpha = 0.6) +
  scale_fill_manual(values=ct_palette)+
  geom_stratum() +
  scale_x_discrete(limits = c("NBM", "NDMM", "PT")) +
  labs(
    x = "Collection",
    y = "AUC (TNFRSF13B)",
    title = "TNFRSF13B AUC across Collection by subset"
  ) +
  theme_classic() +
  theme(
    legend.position = "none",
    axis.text.x = element_text(angle = 90, hjust = 1)
  )

pdf('TNFRSF13B_AUC_alluvial.pdf', width=4)
print(p)
dev.off()

In [ ]:
df <- auc %>% filter(gene == "TNFRSF13C", Collection %in% c("NBM", "NDMM", "PT"), subset %in% c('Transitional B', 'Naive B', 'Memory B', 'PC'))

dfa <- df %>%
  group_by(subset, Collection) %>%
  summarise(AUC = mean(AUC_norm), .groups = "drop")
dfa$subset = factor(dfa$subset, levels=c('Transitional B', 'Naive B', 'Memory B', 'PC'))

p <- ggplot(dfa,
  aes(x = Collection, 
      stratum  = subset,              
      alluvium = subset,              
      y = AUC,                 
      fill  = subset,
      label = subset)) +
  geom_flow(stat = "alluvium",
            lode.guidance = "frontback",
            alpha = 0.6) +
  scale_fill_manual(values=ct_palette)+
  geom_stratum() +
  scale_x_discrete(limits = c("NBM", "NDMM", "PT")) +
  labs(
    x = "Collection",
    y = "AUC (TNFRSF13C)",
    title = "TNFRSF13C AUC across Collection by subset"
  ) +
  theme_classic() +
  theme(
    legend.position = "none",
    axis.text.x = element_text(angle = 90, hjust = 1)
  )

pdf('TNFRSF13C_AUC_alluvial.pdf', width=4)
print(p)
dev.off()

In [ ]:
df <- auc %>% filter(gene == "TNFSF13", Collection %in% c("NBM", "NDMM", "PT"), subset %in% c('CD14 Mc', 'CD16 Mc / TAM', 'Neutrophil', 'cDC'))

dfa <- df %>%
  group_by(subset, Collection) %>%
  summarise(AUC = mean(AUC_norm), .groups = "drop")
dfa$subset = factor(dfa$subset, levels=c('CD14 Mc', 'CD16 Mc / TAM', 'Neutrophil', 'cDC'))

p <- ggplot(dfa,
  aes(x = Collection, 
      stratum  = subset,              
      alluvium = subset,              
      y = AUC,                 
      fill  = subset,
      label = subset)) +
  geom_flow(stat = "alluvium",
            lode.guidance = "frontback",
            alpha = 0.6) +
  scale_fill_manual(values=ct_palette)+
  geom_stratum() +
  scale_x_discrete(limits = c("NBM", "NDMM", "PT")) +
  labs(
    x = "Collection",
    y = "AUC (TNFSF13)",
    title = "TNFSF13 AUC across Collection by subset"
  ) +
  theme_classic() +
  theme(
    legend.position = "none",
    axis.text.x = element_text(angle = 90, hjust = 1)
  )

pdf('TNFSF13_AUC_alluvial.pdf', width=4)
print(p)
dev.off()

In [ ]:
df <- auc %>% filter(gene == "TNFSF13B", Collection %in% c("NBM", "NDMM", "PT"), subset %in% c('CD14 Mc', 'CD16 Mc / TAM', 'Neutrophil', 'cDC'))

dfa <- df %>%
  group_by(subset, Collection) %>%
  summarise(AUC = mean(AUC_norm), .groups = "drop")
dfa$subset = factor(dfa$subset, levels=c('CD14 Mc', 'CD16 Mc / TAM', 'Neutrophil', 'cDC'))

p <- ggplot(dfa,
  aes(x = Collection, 
      stratum  = subset,              
      alluvium = subset,              
      y = AUC,                 
      fill  = subset,
      label = subset)) +
  geom_flow(stat = "alluvium",
            lode.guidance = "frontback",
            alpha = 0.6) +
  scale_fill_manual(values=ct_palette)+
  geom_stratum() +
  scale_x_discrete(limits = c("NBM", "NDMM", "PT")) +
  labs(
    x = "Collection",
    y = "AUC (TNFSF13B)",
    title = "TNFSF13B AUC across Collection by subset"
  ) +
  theme_classic() +
  theme(
    legend.position = "none",
    axis.text.x = element_text(angle = 90, hjust = 1)
  )

pdf('TNFSF13B_AUC_alluvial.pdf', width=4)
print(p)
dev.off()